In [1]:
library("lme4")
library("ggplot2")
library("dplyr")
library(MASS)
library("BayesFactor")
library("car")
library("scales")
library("lmerTest")
library("MuMIn")
library("plyr")
library("rstatix")
library("ggpubr")
library("knitr")
library("corrplot")
library("RColorBrewer")

Loading required package: Matrix


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘MASS’


The following object is masked from ‘package:dplyr’:

    select


Loading required package: coda

************
Welcome to BayesFactor 0.9.12-4.2. If you have questions, please contact Richard Morey (richarddmorey@gmail.com).

Type BFManual() to open the manual.
************

Loading required package: carData

Registered S3 methods overwritten by 'car':
  method                          from
  influence.merMod                lme4
  cooks.distance.influence.merMod lme4
  dfbeta.influence.merMod         lme4
  dfbetas.influence.merMod        lme4


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode



Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

   

In [2]:
#Load the data => Here, we are filtering the cases in which subjects had a MuIFI of 0
FinalPath<-'../ToInput/R_HumMod_MS17_notemp.txt'
data <- read.delim(FinalPath, sep="\t", header=T, stringsAsFactors=F)
NoTemp_N <- nrow(data %>% filter(Temperature=="no"))
YesTemp_N <- nrow(data %>% filter(Temperature=="yes"))
data <- data %>% filter(!Amplitude==0)
dataTest <- data
data$lCV <- log(data$CV)

In [3]:
table(data$Temperature, data$GameSpeed)

       
         MMM  MSM  SMS  SSS
  Human  273  286  282  273
  no    1499 1490 1495 1499
  yes   1499 1491 1498 1496

In [4]:
## Gives count, mean, standard deviation, standard error of the mean, and confidence interval (default 95%).
##   data: a data frame.
##   measurevar: the name of a column that contains the variable to be summariezed
##   groupvars: a vector containing names of columns that contain grouping variables
##   na.rm: a boolean that indicates whether to ignore NA's
##   conf.interval: the percent range of the confidence interval (default is 95%)
summarySE <- function(data=NULL, measurevar, groupvars=NULL, na.rm=FALSE,
                      conf.interval=.95, .drop=TRUE) {
    library(plyr)

    # New version of length which can handle NA's: if na.rm==T, don't count them
    length2 <- function (x, na.rm=FALSE) {
        if (na.rm) sum(!is.na(x))
        else       length(x)
    }

    # This does the summary. For each group's data frame, return a vector with
    # N, mean, and sd
    datac <- ddply(data, groupvars, .drop=.drop,
      .fun = function(xx, col) {
        c(N    = length2(xx[[col]], na.rm=na.rm),
          mean = mean   (xx[[col]], na.rm=na.rm),
          sd   = sd     (xx[[col]], na.rm=na.rm)
        )
      },
      measurevar
    )

    # Rename the "mean" column    
    datac <- rename(datac, c("mean" = measurevar))

    datac$se <- datac$sd / sqrt(datac$N)  # Calculate standard error of the mean

    # Confidence interval multiplier for standard error
    # Calculate t-statistic for confidence interval: 
    # e.g., if conf.interval is .95, use .975 (above/below), and use df=N-1
    ciMult <- qt(conf.interval/2 + .5, datac$N-1)
    datac$ci <- datac$se * ciMult

    return(datac)
}

In [5]:
# Multiple plot function
#
# ggplot objects can be passed in ..., or to plotlist (as a list of ggplot objects)
# - cols:   Number of columns in layout
# - layout: A matrix specifying the layout. If present, 'cols' is ignored.
#
# If the layout is something like matrix(c(1,2,3,3), nrow=2, byrow=TRUE),
# then plot 1 will go in the upper left, 2 will go in the upper right, and
# 3 will go all the way across the bottom.
#
multiplot <- function(..., plotlist=NULL, file, cols=1, layout=NULL) {
  library(grid)

  # Make a list from the ... arguments and plotlist
  plots <- c(list(...), plotlist)

  numPlots = length(plots)

  # If layout is NULL, then use 'cols' to determine layout
  if (is.null(layout)) {
    # Make the panel
    # ncol: Number of columns of plots
    # nrow: Number of rows needed, calculated from # of cols
    layout <- matrix(seq(1, cols * ceiling(numPlots/cols)),
                    ncol = cols, nrow = ceiling(numPlots/cols))
  }

 if (numPlots==1) {
    print(plots[[1]])

  } else {
    # Set up the page
    grid.newpage()
    pushViewport(viewport(layout = grid.layout(nrow(layout), ncol(layout))))

    # Make each plot, in the correct location
    for (i in 1:numPlots) {
      # Get the i,j matrix positions of the regions that contain this subplot
      matchidx <- as.data.frame(which(layout == i, arr.ind = TRUE))

      print(plots[[i]], vp = viewport(layout.pos.row = matchidx$row,
                                      layout.pos.col = matchidx$col))
    }
  }
}

# Compute r and RMSE

In [6]:
#RMSE function definition
RMSE = function(mod, obs){
  sqrt(mean((mod - obs)^2))
}

RSS = function(mod,obs){
    sum((mod - obs)^2)
}

BIC = function(RSS,n,k){
    n*log(RSS/n)+k*log(n)
}

#Filtered data set
filt_data <- data %>% filter(!Temperature=="yes")

In [7]:
#1) Performance results
muTab<-aggregate(filt_data$Points, by=list(filt_data$Agent,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
colnames(muTab)<-c("Agent","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Agent=="Human" & muTab$GameNb==i),]$Value
}
M_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M_Mat[i,] <- muTab[which(muTab$Agent=="Model" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M_Mat) <- 1:15

H_vec <- c(H_Mat)
M_vec <- c(M_Mat)
Perf_r <- cor(H_vec,M_vec,method = "pearson")
Perf_RMSE <- RMSE(M_vec,H_vec)
Perf_RSS <- RSS(M_vec,H_vec)
Perf_RMSE
Perf_RSS

[1] 311.939

[1] 5838357

In [8]:
#2) Entropy results
muTab<-aggregate(filt_data$Entropy, by=list(filt_data$Agent,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
colnames(muTab)<-c("Agent","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Agent=="Human" & muTab$GameNb==i),]$Value
}
M_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M_Mat[i,] <- muTab[which(muTab$Agent=="Model" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M_Mat) <- 1:15

H_vec <- c(H_Mat)
M_vec <- c(M_Mat)
Ent_r <- cor(H_vec,M_vec,method = "pearson")
Ent_RMSE <- RMSE(M_vec,H_vec)
Ent_RSS <- RSS(M_vec,H_vec)
Ent_RMSE
Ent_RSS

[1] 0.1407784

[1] 1.189114

In [9]:
#3) Log CV ISI results
muTab<-aggregate(filt_data$lCV, by=list(filt_data$Agent,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
colnames(muTab)<-c("Agent","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Agent=="Human" & muTab$GameNb==i),]$Value
}
M_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M_Mat[i,] <- muTab[which(muTab$Agent=="Model" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M_Mat) <- 1:15

H_vec <- c(H_Mat)
M_vec <- c(M_Mat)
lcv_r <- cor(H_vec,M_vec,method = "pearson")
lcv_RMSE <- RMSE(M_vec,H_vec)
lcv_RSS <- RSS(M_vec,H_vec)
lcv_RMSE
lcv_RSS

[1] 0.3738097

[1] 8.38402

In [10]:
#4) Periodicity results
muTab<-aggregate(filt_data$Periodicity, by=list(filt_data$Agent,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
colnames(muTab)<-c("Agent","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Agent=="Human" & muTab$GameNb==i),]$Value
}
M_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M_Mat[i,] <- muTab[which(muTab$Agent=="Model" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M_Mat) <- 1:15

H_vec <- c(H_Mat)
M_vec <- c(M_Mat)
Per_r <- cor(H_vec,M_vec,method = "pearson")
Per_RMSE <- RMSE(M_vec,H_vec)
Per_RSS <- RSS(M_vec,H_vec)
Per_RMSE
Per_RSS

[1] 65.5688

[1] 257956.1

In [11]:
#5) Regularity results
muTab<-aggregate(filt_data$Amplitude, by=list(filt_data$Agent,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
colnames(muTab)<-c("Agent","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Agent=="Human" & muTab$GameNb==i),]$Value
}
M_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M_Mat[i,] <- muTab[which(muTab$Agent=="Model" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M_Mat) <- 1:15

H_vec <- c(H_Mat)
M_vec <- c(M_Mat)
Reg_r <- cor(H_vec,M_vec,method = "pearson")
Reg_RMSE <- RMSE(M_vec,H_vec)
Reg_RSS <- RSS(M_vec,H_vec)
Reg_RMSE
Reg_RSS

[1] 0.03810177

[1] 0.08710469

In [12]:
#6) Resets results
muTab<-aggregate(filt_data$Resets, by=list(filt_data$Agent,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
colnames(muTab)<-c("Agent","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Agent=="Human" & muTab$GameNb==i),]$Value
}
M_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M_Mat[i,] <- muTab[which(muTab$Agent=="Model" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M_Mat) <- 1:15

H_vec <- c(H_Mat)
M_vec <- c(M_Mat)
Res_r <- cor(H_vec,M_vec,method = "pearson")
Res_RMSE <- RMSE(M_vec,H_vec)
Res_RSS <- RSS(M_vec,H_vec)
Res_RMSE
Res_RSS

[1] 5.086502

[1] 1552.35

In [13]:
#7) Deflations results
muTab<-aggregate(filt_data$Deflations, by=list(filt_data$Agent,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
colnames(muTab)<-c("Agent","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Agent=="Human" & muTab$GameNb==i),]$Value
}
M_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M_Mat[i,] <- muTab[which(muTab$Agent=="Model" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M_Mat) <- 1:15

H_vec <- c(H_Mat)
M_vec <- c(M_Mat)
Def_r <- cor(H_vec,M_vec,method = "pearson")
Def_RMSE <- RMSE(M_vec,H_vec)
Def_RSS <- RSS(M_vec,H_vec)
Def_RMSE
Def_RSS

[1] 3.715538

[1] 828.3133

In [14]:
#8) Misses results
muTab<-aggregate(filt_data$Misses, by=list(filt_data$Agent,filt_data$GameSpeed,filt_data$GameNb), FUN=mean)
colnames(muTab)<-c("Agent","GameSpeed","GameNb","Value")
H_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    H_Mat[i,] <- muTab[which(muTab$Agent=="Human" & muTab$GameNb==i),]$Value
}
M_Mat <- matrix(, nrow = 15, ncol = 4)
for (i in 1:15)
{
    M_Mat[i,] <- muTab[which(muTab$Agent=="Model" & muTab$GameNb==i),]$Value
}
colnames(H_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(H_Mat) <- 1:15
colnames(M_Mat) <- c("MMM","MSM","SMS","SSS")
rownames(M_Mat) <- 1:15

H_vec <- c(H_Mat)
M_vec <- c(M_Mat)
Mis_r <- cor(H_vec,M_vec,method = "pearson")
Mis_RMSE <- RMSE(M_vec,H_vec)
Mis_RSS <- RSS(M_vec,H_vec)
Mis_RMSE
Mis_RSS

[1] 6.506537

[1] 2540.101

In [15]:
#BIC measure
n=60
k=0
Points_BIC <- BIC(Perf_RSS,n,k)
Points_BIC
Ent_BIC <- BIC(Ent_RSS,n,k)
Ent_BIC
lCV_BIC <- BIC(lcv_RSS,n,k)
lCV_BIC
Per_BIC <- BIC(Per_RSS,n,k)
Per_BIC
Reg_BIC <- BIC(Reg_RSS,n,k)
Reg_BIC
Res_BIC <- BIC(Res_RSS,n,k)
Res_BIC
Def_BIC <- BIC(Def_RSS,n,k)
Def_BIC
Mis_BIC <- BIC(Mis_RSS,n,k)
Mis_BIC

[1] 689.1369

[1] -235.2682

[1] -118.081

[1] 501.972

[1] -392.0993

[1] 195.1909

[1] 157.5028

[1] 224.7369